# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center> **Master1** </center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [232]:
import sqlite3    # Python interface for SQLite databases

In [233]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [234]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
achats:
  0: NB(NUM)
  1: NV(NUM)
  2: LIEU(TEXT)
  3: DATES(NUM)
  4: QTE(NUM)
acheteurs:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
lieu:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producteur:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
produits:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
vins:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
vins1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
vins2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


From this point we will use __%%sql__ magic

In [235]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [236]:
%%sql SELECT *
FROM Master1
LIMIT 100;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [237]:
%%sql SELECT *
FROM Master2;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,DATES,LIEU,QTE,NB,NOM,PRENOM,TYPE,REGION
None,None,None,None,None,None,None,11,Breton,Andre,petit,None
None,None,None,None,None,None,None,13,Barthes,Roland,moyen,None
None,None,None,None,None,None,None,16,Balzac,Honore de,moyen,None
None,None,None,None,None,None,None,18,Celine,Louis Ferdinand,gros,None
None,None,None,None,None,None,None,20,Chateaubriand,Francois-Rene de,moyen,None
None,None,None,None,None,None,None,21,Corbiere,Tristan,petit,None
None,None,None,None,None,None,None,23,Corneille,Pierre,petit,None
None,None,None,None,None,None,None,25,Char,Rene,petit,None
None,None,None,None,None,None,None,27,Dumas,Alexandre,gros,None
None,None,None,None,None,None,None,29,Fournier,Alain,petit,None


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

<img src="./question1.jpeg">
<img src="./question1.2.jpeg">

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

<img src="./question2.jpeg">

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [238]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [239]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
achats:
  0: NB(NUM)
  1: NV(NUM)
  2: LIEU(TEXT)
  3: DATES(NUM)
  4: QTE(NUM)
acheteurs:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
dummy:
  0: DEGRE(NUM)
lieu:
  0: LIEU(TEXT)
  1: REGION(TEXT)
producteur:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
produits:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
vins:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
vins1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
vins2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


In [240]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [241]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [242]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [243]:
%%sql DROP TABLE IF EXISTS producteur;

CREATE TABLE producteur AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1 WHERE NP IS NOT NULL;

SELECT *
FROM producteur;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


In [244]:
%%sql DROP TABLE IF EXISTS vins1;

CREATE TABLE vins1 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1 WHERE NV IS NOT NULL;


SELECT *
FROM vins1;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [245]:
%%sql DROP TABLE IF EXISTS produits;

CREATE TABLE produits AS
SELECT NV, NP, QTE
FROM MASTER1 WHERE NV IS NOT NULL AND NP IS NOT NULL;


SELECT *
FROM produits;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,NP,QTE
1,1,300
1,73,1
2,5,100
3,1,400
4,10,35
5,30,46
6,42,300
7,98,60
8,90,12
10,98,100


#### Exercise 1.4

Create the new tables from Master2:

In [246]:
%%sql DROP TABLE IF EXISTS acheteurs;

CREATE TABLE acheteurs AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2 WHERE NB IS NOT NULL;


SELECT *
FROM acheteurs;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


In [247]:
%%sql DROP TABLE IF EXISTS vins2;

CREATE TABLE vins2 AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER2 WHERE NV IS NOT NULL;


SELECT *
FROM vins2;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [248]:
%%sql DROP TABLE IF EXISTS lieu;

CREATE TABLE lieu AS
SELECT DISTINCT LIEU, REGION
FROM MASTER2 WHERE LIEU IS NOT NULL;


SELECT *
FROM lieu;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR
MERCUREY,BOURGOGNE-FRANCHE-COMTÉ
TOULOUSE,OCCITANIE
SENS,BOURGOGNE-FRANCHE-COMTÉ
LILLE,HAUTS-DE-FRANCE
NANCY,GRAND EST


In [249]:
%%sql DROP TABLE IF EXISTS achats;

CREATE TABLE achats AS
SELECT NB, NV, LIEU, DATES, QTE
FROM MASTER2 WHERE NB IS NOT NULL AND NV IS NOT NULL;


SELECT *
FROM achats;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NV,LIEU,DATES,QTE
2,1,BORDEAUX,1977-11-02,33
44,1,PARIS,2015-10-16,1
45,1,RENNES,1983-12-31,1
48,2,LYON,1983-12-25,2
7,3,NICE,1978-11-01,6
49,3,MERCUREY,1987-12-24,5
8,4,TOULOUSE,1982-11-05,12
44,4,PARIS,2015-10-16,6
50,4,SENS,1981-06-29,4
2,5,MERCUREY,1984-11-04,50


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [250]:
%%sql SELECT DISTINCT(TYPE) as Types_of_buveurs
FROM acheteurs;

 * sqlite:///wine.db
Done.


Types_of_buveurs
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [251]:
%%sql 
SELECT CRU, REGION 
FROM produits JOIN vins1 ON produits.NV=vins1.NV 
JOIN producteur on producteur.NP=produits.NP 
WHERE CRU="Pommard"  or CRU="Brouilly";

 * sqlite:///wine.db
Done.


CRU,REGION
Pommard,Bourgogne
Pommard,Rhone
Brouilly,Bourgogne


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [252]:
%%sql 
SELECT REGION 
FROM produits JOIN vins1 ON produits.NV=vins1.NV 
JOIN producteur on producteur.NP=produits.NP 
WHERE CRU="Pommard"  
INTERSECT
SELECT REGION 
FROM produits JOIN vins1 ON produits.NV=vins1.NV 
JOIN producteur on producteur.NP=produits.NP 
WHERE CRU="Brouilly";

 * sqlite:///wine.db
Done.


REGION
Bourgogne


#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [253]:
%%sql 
SELECT CRU,MILL,SUM(QTE)
FROM achats JOIN vins2 on achats.NV=vins2.NV
GROUP BY CRU, MILL

 * sqlite:///wine.db
Done.


CRU,MILL,SUM(QTE)
Arbois,1980,8
Auxey Duresses,1914,80
Beaujolais Primeur,1983,7
Beaujolais Villages,1975,10
Beaujolais Villages,1976,120
Beaujolais Villages,1978,130
Beaujolais Villages,1979,520
Chapelle Chambertin,1973,30
Chateau Corton Grancey,1980,4
Chenas,1984,1


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [254]:
%%sql 
SELECT NV, COUNT(DISTINCT(NP)) as Number_of_producers
FROM produits GROUP BY NV HAVING COUNT(DISTINCT(NP))>3;

 * sqlite:///wine.db
Done.


NV,Number_of_producers
45,5
78,5
89,4
98,5


#### Exercise 2.6

Which producers have not produced any wine?

In [255]:
%%sql 
SELECT p1.NP, p1.NOM, p1.PRENOM
FROM producteur as p1
EXCEPT
SELECT p2.NP, p2.NOM, p2.PRENOM
FROM producteur as p2
JOIN produits on p2.NP=produits.NP

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM
3,Six,Paul
6,Marmagne,Bernard
8,Lioger d'Harduy,Gabriel
16,Barbin,Bernard
17,Faiveley,Guy
18,Tramier,Jean
19,Dupaquier,Roger
20,Lamy,Jean
21,Cornu,Edmond
26,Violot,Gilbert


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [256]:
%%sql DROP VIEW IF EXISTS question7;
CREATE VIEW question7 AS
SELECT DISTINCT a2.NB, a2.NOM, a2.PRENOM
FROM achats as a1 JOIN acheteurs as a2 ON a1.NB=a2.NB
JOIN vins2 on vins2.NV=a1.NV
WHERE vins2.MILL=1980;
SELECT * FROM question7;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM
2,Artaud,Antonin
44,Gide,Andre
45,Giono,Jean
8,Aragon,Louis
50,Lautreamont,None
61,Mallarme,Stephane


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [257]:
%%sql DROP VIEW IF EXISTS question8;
CREATE VIEW question8 AS
SELECT DISTINCT a2.NB, a2.NOM, a2.PRENOM
FROM achats as a1 JOIN acheteurs as a2 ON a1.NB=a2.NB
JOIN vins2 on vins2.NV=a1.NV
EXCEPT
SELECT DISTINCT NB, NOM, PRENOM
FROM question7;
SELECT * FROM question8;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM
1,Aristote,None
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
9,Ajar,Emile
10,Andersen,Yann
12,Bataille,Georges
14,Bory,Jean Louis


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [258]:
%%sql DROP VIEW IF EXISTS question9;
CREATE VIEW question9 AS
SELECT DISTINCT NB, NOM, PRENOM
FROM question7
EXCEPT
SELECT DISTINCT a2.NB, a2.NOM, a2.PRENOM
FROM achats as a1 JOIN acheteurs as a2 ON a1.NB=a2.NB
JOIN vins2 on vins2.NV=a1.NV
WHERE vins2.MILL!=1980;
SELECT * FROM question9;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM
44,Gide,Andre
45,Giono,Jean
50,Lautreamont,None


#### Exercise 2.10

List all wines from 1980

In [259]:
%%sql DROP VIEW IF EXISTS question10;
CREATE VIEW question10 AS
SELECT NV, CRU, MILL, DEGRE
FROM vins1 WHERE MILL=1980;
SELECT * FROM question10;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5
4,Mercurey,1980,10.9
16,Meursault,1980,12.1
20,Cote de Brouilly,1980,12.1
26,Chateau Corton Grancey,1980,None
28,Volnay,1980,11
43,Fleurie,1980,11.4
74,Arbois,1980,12
78,Etoile,1980,12
79,Seyssel,1980,11


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [260]:
%%sql DROP VIEW IF EXISTS question11;
CREATE VIEW question11 AS
SELECT achats.NV, CRU, MILL, DEGRE
FROM achats JOIN question10 ON question10.NV=achats.NV WHERE NB=2;
SELECT * FROM question11;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,MILL,DEGRE
1,Mercurey,1980,11.5


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [261]:
%%sql 
SELECT DISTINCT acheteurs.NB, acheteurs.NOM, acheteurs.PRENOM
FROM acheteurs JOIN achats as a1 ON a1.NB=acheteurs.NB
WHERE NOT EXISTS
(SELECT question10.NV FROM question10
EXCEPT
SELECT a2.NV FROM achats as a2
WHERE a2.NB=a1.NB);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
44,Gide,Andre
